# The Knapsack Problem


## Summary

We have the opportunity to put several valuables in a backpack, trying to maximize the value of the objects

The backpack only supports a weight of 2.5kg and a volume of 2000 cm$^3$

What objects should we take?

![uc3m](KnapsackData.png)



    






### Mathematical formulation

## Formulation with Pyomo



### The data

We are going go read the data directly from an Excel file


In [3]:
from pyomo.environ import *

import pandas as pd
#from pyomo.environ import *
#from pyomo.opt import SolverFactory

# Import data using pandas
df = pd.read_excel('knapsack.xlsx')  
print(df)

# Data preparation
max_volume = 2000
max_weight = 2500
 
item = list(set(df['Item']))

price = df.set_index(['Item'])['Price'].to_dict()
volume = df.set_index(['Item'])['Volume'].to_dict()
weight = df.set_index(['Item'])['Weight'].to_dict()
available = df.set_index(['Item'])['Available'].to_dict()

print(price)

          Item  Price  Volume  Weight  Available
0        Chest     50    1000    2000          1
1         Ring      5       2      20         10
2     Necklace      3      10     300          1
3       Mirror     20     500    1000          1
4     Bracelet     16      15     300         15
5         Ruby      5       3      75          1
6       Parfum      1     100     100          1
7      Diamond     30       5      50          1
8  Gold goblet     12     250     500          1
9        Spice     40     100     100          1
{'Chest': 50, 'Ring': 5, 'Necklace': 3, 'Mirror': 20, 'Bracelet': 16, 'Ruby': 5, 'Parfum': 1, 'Diamond': 30, 'Gold goblet': 12, 'Spice': 40}


### The model

In [4]:
print(price[item[0]])

20


In [5]:
model = ConcreteModel()


### Insert your code here

from pyomo.environ import *


model.item = Set(initialize=item, doc='model_car' )

model.x = Var(model.item, within = NonNegativeIntegers)


model.kg = Param(initialize=2500)
model.v = Param(initialize=2000)

def obj_expression(model): 
    return sum(model.x[i]*price[i] for i in model.item)

model.OBJ = Objective(rule=obj_expression, sense=maximize)



def max_number(model, index):
    return model.x[index] <= available[index]
model.max_number = Constraint(model.item, rule=max_number)



def kg_cons(model):
    return sum(model.x[i]*weight[i] for i in model.item) <= model.kg
model.kg_cons = Constraint(item, rule=kg_cons)


def volume_cons(model):
    return sum(model.x[i]*volume[i] for i in model.item) <= model.v
model.volume_cons = Constraint(item, rule=volume_cons)



In [6]:
# Solution
# opt = SolverFactory('gurobi')
opt = SolverFactory('glpk')

results = opt.solve(model)  # solve the model with the selected solver
 
model.display()


Model unknown

  Variables:
    x : Size=10, Index=item
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
           Bracelet :     0 :   7.0 :  None : False : False : NonNegativeIntegers
              Chest :     0 :   0.0 :  None : False : False : NonNegativeIntegers
            Diamond :     0 :   1.0 :  None : False : False : NonNegativeIntegers
        Gold goblet :     0 :   0.0 :  None : False : False : NonNegativeIntegers
             Mirror :     0 :   0.0 :  None : False : False : NonNegativeIntegers
           Necklace :     0 :   0.0 :  None : False : False : NonNegativeIntegers
             Parfum :     0 :   0.0 :  None : False : False : NonNegativeIntegers
               Ring :     0 :  10.0 :  None : False : False : NonNegativeIntegers
               Ruby :     0 :   0.0 :  None : False : False : NonNegativeIntegers
              Spice :     0 :   1.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    OBJ : Size=1, Index=None, Active=Tru

### Interpretation

Take 7 bracelets, 1 diamond, 10 rings and 1 spice. Total value is 232.